In [1]:
pip install bitarray

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install struct

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement struct (from versions: none)
ERROR: No matching distribution found for struct


In [12]:
# Step 1: Import necessary libraries
import numpy as np
import wave
import struct
import bitarray
import os

In [4]:
# Step 2: Define functions for Rice encoding and decoding
def rice_encode(data, k):
    encoded = bitarray.bitarray()
    for value in data:
        q, r = divmod(value, 2 ** k)
        encoded.extend('1' * q + '0' + format(r, f'0{k}b'))
    return encoded


In [5]:
def rice_decode(encoded, k):
    decoded = []
    bits = iter(encoded)
    while True:
        q = 0
        try:
            while next(bits):
                q += 1
            r = int(''.join([str(next(bits)) for _ in range(k)]), 2)
            decoded.append((2 ** k) * q + r)
        except StopIteration:
            break
    return np.array(decoded, dtype=np.int16)

In [6]:
# Step 3: Create functions to read and write WAV files
def read_wav_file(file_path):
    with wave.open(file_path, 'rb') as wav_file:
        n_channels, sampwidth, framerate, n_frames, _, _ = wav_file.getparams()
        frames = wav_file.readframes(n_frames)
        data = struct.unpack(f'{n_frames * n_channels}h', frames)
        return np.array(data, dtype=np.int16)


In [17]:
def write_wav_file(filename, data, n_channels, sampwidth, framerate):
    data = np.clip(data, -32768, 32767).astype(np.int16)  # Clip the data to the valid range
    with wave.open(filename, 'wb') as wav_file:
        wav_file.setparams((n_channels, sampwidth, framerate, len(data), 'NONE', 'not compressed'))
        wav_file.writeframes(data.tobytes())



In [18]:
# Step 4: Implement the encoding and decoding process
def encode_and_decode(input_file, encoded_file, decoded_file, k):
    # Read the input WAV file
    data = read_wav_file(input_file)
    
    # Encode the data using Rice coding
    encoded_data = rice_encode(data, k)
    
    # Save the encoded data to a file
    with open(encoded_file, 'wb') as f:
        encoded_data.tofile(f)
    
    # Load the encoded data from the file
    with open(encoded_file, 'rb') as f:
        loaded_encoded_data = bitarray.bitarray()
        loaded_encoded_data.fromfile(f)
    
    # Decode the encoded data
    decoded_data = rice_decode(loaded_encoded_data, k)
    
    # Write the decoded data to a new WAV file
    with wave.open(input_file, 'rb') as wav_file:
        n_channels, sampwidth, framerate, _, _, _ = wav_file.getparams()
    write_wav_file(decoded_file, decoded_data, n_channels, sampwidth, framerate)

In [19]:
# Step 5: Analyze the effectiveness of the Rice coding algorithm
input_files = ['Sound1.wav', 'Sound2.wav']
ks = [4, 2]


In [20]:
for input_file in input_files:
    original_size = os.path.getsize(input_file)
    for k in ks:
        encoded_file = input_file.replace('.wav', f'_Enc_K{k}.ex2')
        decoded_file = input_file.replace('.wav', f'_Enc_Dec_K{k}.wav')
                # Encode and decode the input file
        encode_and_decode(input_file, encoded_file, decoded_file, k)
        
        # Calculate the compressed file size and compression rate
        compressed_size = os.path.getsize(encoded_file)
        compression_rate = (original_size - compressed_size) / original_size * 100
        
        # Print the results
        print(f'{input_file} (K = {k}):')
        print(f'  Original size: {original_size} bytes')
        print(f'  Compressed size: {compressed_size} bytes')
        print(f'  Compression rate: {compression_rate:.2f}%')
        print()


Sound1.wav (K = 4):
  Original size: 1002088 bytes
  Compressed size: 1733251 bytes
  Compression rate: -72.96%

Sound1.wav (K = 2):
  Original size: 1002088 bytes
  Compressed size: 5904627 bytes
  Compression rate: -489.23%

Sound2.wav (K = 4):
  Original size: 1008044 bytes
  Compressed size: 10172734 bytes
  Compression rate: -909.16%

Sound2.wav (K = 2):
  Original size: 1008044 bytes
  Compressed size: 39667254 bytes
  Compression rate: -3835.07%



In [21]:
def delta_encode(data):
    delta_encoded = np.diff(data, prepend=data[0])
    return delta_encoded

def delta_decode(delta_encoded):
    decoded = np.cumsum(delta_encoded)
    return decoded


In [22]:
def encode_and_decode_with_delta(input_file, encoded_file, decoded_file, k):
    # Read the input WAV file
    data = read_wav_file(input_file)
    
    # Apply delta encoding
    delta_encoded_data = delta_encode(data)
    
    # Encode the data using Rice coding
    encoded_data = rice_encode(delta_encoded_data, k)
    
    # Save the encoded data to a file
    with open(encoded_file, 'wb') as f:
        encoded_data.tofile(f)
    
    # Load the encoded data from the file
    with open(encoded_file, 'rb') as f:
        loaded_encoded_data = bitarray.bitarray()
        loaded_encoded_data.fromfile(f)
    
    # Decode the encoded data using Rice coding
    decoded_delta_data = rice_decode(loaded_encoded_data, k)
    
    # Apply delta decoding
    decoded_data = delta_decode(decoded_delta_data)
    
    # Write the decoded data to a new WAV file
    with wave.open(input_file, 'rb') as wav_file:
        n_channels, sampwidth, framerate, _, _, _ = wav_file.getparams()
    write_wav_file(decoded_file, decoded_data, n_channels, sampwidth, framerate)


In [23]:
for input_file in input_files:
    original_size = os.path.getsize(input_file)
    for k in ks:
        encoded_file = input_file.replace('.wav', f'_Enc_K{k}_Delta.ex2')
        decoded_file = input_file.replace('.wav', f'_Enc_Dec_K{k}_Delta.wav')
        
        # Encode and decode the input file using delta coding and Rice coding
        encode_and_decode_with_delta(input_file, encoded_file, decoded_file, k)
        
        # Calculate the compressed file size and compression rate
        compressed_size = os.path.getsize(encoded_file)
        compression_rate = (original_size - compressed_size) / original_size * 100
        
        # Print the results
        print(f'{input_file} with Delta coding (K = {k}):')
        print(f'  Original size: {original_size} bytes')
        print(f'  Compressed size: {compressed_size} bytes')
        print(f'  Compression rate: {compression_rate:.2f}%')
        print()


Sound1.wav with Delta coding (K = 4):
  Original size: 1002088 bytes
  Compressed size: 445108 bytes
  Compression rate: 55.58%

Sound1.wav with Delta coding (K = 2):
  Original size: 1002088 bytes
  Compressed size: 744827 bytes
  Compression rate: 25.67%

Sound2.wav with Delta coding (K = 4):
  Original size: 1008044 bytes
  Compressed size: 14332986 bytes
  Compression rate: -1321.86%

Sound2.wav with Delta coding (K = 2):
  Original size: 1008044 bytes
  Compressed size: 56308140 bytes
  Compression rate: -5485.88%

